Carregando o pandas, numpy e imprimindo a versão do pandas

In [16]:
import numpy as np
import pandas as pd
import gzip
import io
import requests

PANDA_VERSION = pd.__version__
PROPOSICAO_JSON_DS_FILE = 'https://github.com/bozoh/meu-legislador/raw/master/data/ds/proposicoes.json.gz'
DEPUTADOS_JSON_DS_FILE = 'https://github.com/bozoh/meu-legislador/raw/master/data/ds/deputados.json.gz'

In [12]:
PANDA_VERSION

'0.22.0'

Carregando os dados de todas as proposições (PL, PLP, PEC, PLV, PDC, MPV) da câmara dos deputados 
a partit de 1/1/1984


In [59]:
web_response = requests.get(PROPOSICAO_JSON_DS_FILE, timeout=30, stream=True)
f = io.BytesIO(web_response.content)

with gzip.GzipFile(fileobj=f) as fh:
    df_prep = pd.read_json(fh, orient='split')

Carregando dados dos depudados

In [60]:
web_response = requests.get(DEPUTADOS_JSON_DS_FILE, timeout=30, stream=True)
f = io.BytesIO(web_response.content)

with gzip.GzipFile(fileobj=f) as fh:
    df_dep = pd.read_json(fh, orient='split')

In [ ]:
Nesse estudo só quero o nome dos deputados

In [61]:
df_dep = df_dep[['id', 'nomeCivil']]
df_dep

,id,nomeCivil
0,129037,GABRIEL GUIMARÃES DE ANDRADE
1,141335,JOSÉ ROBERTO OLIVEIRA FARO
2,73768,DILCEU JOÃO SPERAFICO
3,73772,HERMES PARCIANELLO
4,73778,LUIZ CARLOS JORGE HAULY
5,73781,NELSON MEURER
6,73786,REINHOLD STEPHANES
7,141372,AELTON JOSÉ DE FREITAS
8,141383,ANTONIO CARLOS MARTINS DE BULHÕES
9,141384,ANTÔNIO CARLOS VALADARES FILHO


Acrescentando a coluna idAutor, obtido pela coluna uriAutores, que no final tem o id do deputado

In [62]:
df_prep['idAutor'] = df_prep.uriAutores.map(lambda x: int(x.replace('https://dadosabertos.camara.leg.br/api/v2/deputados/','')) if x else np.nan)

Acrescentando o nome do autor nas proposições

In [63]:
df_prep=pd.merge(left=df_prep, right=df_dep, left_on='idAutor', right_on='id')
df_prep

,ano,dataApresentacao,descricaoTipo,ementa,ementaDetalhada,id_x,idTipo,idTipoAutor,justificativa,keywords,...,uriOrgaoNumerador,uriPropAnterior,uriPropPosterior,uriPropPrincipal,uriUltimoRelator,urlInteiroTeor,urnFinal,idAutor,id_y,nomeCivil
0,2011,2011-10-19T21:52,Projeto de Lei,"Acresce dispositivo à Lei nº 10.826, de 2003 -...",NaN,524317,139,10000,NaN,"Alteração, Estatuto do Desarmamento, proibição...",...,NaN,NaN,NaN,None,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,160660,160660,MARCELO THEODORO DE AGUIAR
1,2011,2011-11-23T21:00,Projeto de Lei,"Altera a Lei nº 10.098, de 19 de dezembro de 2...",NaN,528513,139,10000,NaN,"Alteração, Lei da Acessibilidade, ampliação, e...",...,NaN,NaN,NaN,https://dadosabertos.camara.leg.br/api/v2/prop...,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,160660,160660,MARCELO THEODORO DE AGUIAR
2,2011,2011-11-23T21:01,Projeto de Lei,Institui a Semana Evangélica em todo territóri...,NaN,528514,139,10000,NaN,"Criação, Semana Evangélica, comemoração, outubro.",...,NaN,NaN,NaN,None,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,160660,160660,MARCELO THEODORO DE AGUIAR
3,2016,2016-09-20T15:19,Projeto de Lei,Dispõe sobre obrigatoriedade do pagamento da ...,NaN,2112249,139,10000,NaN,"Obrigatoriedade, consumidor, pagamento, gorjet...",...,NaN,NaN,NaN,https://dadosabertos.camara.leg.br/api/v2/prop...,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,160660,160660,MARCELO THEODORO DE AGUIAR
4,2017,2017-03-09T17:09,Projeto de Lei,"Revoga as Leis n°s 10.826, de 22 de dezembro d...",NaN,2124983,139,10000,NaN,"Revogação, Estatuto do Desarmamento. _Revogaçã...",...,NaN,NaN,NaN,None,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,160660,160660,MARCELO THEODORO DE AGUIAR
5,2013,2013-10-01T22:32,Projeto de Lei,Dispõe sobre o auxílio à mãe solteira em situa...,NaN,594251,139,10000,NaN,"Auxílio, mãe solteira, situação, pobreza.",...,NaN,NaN,NaN,None,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,160660,160660,MARCELO THEODORO DE AGUIAR
6,2011,2011-03-16T21:23,Projeto de Lei,Disciplina o inciso III do art. 19 e o caput d...,NaN,494905,139,10000,NaN,"Proibição, poder público, ato administrativo, ...",...,NaN,NaN,NaN,None,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,160660,160660,MARCELO THEODORO DE AGUIAR
7,2011,2011-03-16T21:24,Projeto de Lei,"Autoriza o Poder Público a criar o Programa ""V...",NaN,494907,139,10000,NaN,"Criação, Programa ""Viver de bem"" - Sem Estigma...",...,NaN,NaN,NaN,None,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,160660,160660,MARCELO THEODORO DE AGUIAR
8,2016,2016-10-10T17:33,Projeto de Lei,"Altera o Decreto-Lei nº 911, de 1º. de outubro...",NaN,2113534,139,10000,NaN,"Alteração, Lei de Alienação Fiduciária, deved...",...,NaN,NaN,NaN,https://dadosabertos.camara.leg.br/api/v2/prop...,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,160660,160660,MARCELO THEODORO DE AGUIAR
9,2016,2016-10-10T17:33,Projeto de Lei,Dispõe sobre as condições de vida e de trabalh...,NaN,2113535,139,10000,NaN,"Regulamentação, condições de trabalho, profiss...",...,NaN,NaN,NaN,https://dadosabertos.camara.leg.br/api/v2/prop...,NaN,http://www.camara.gov.br/proposicoesWeb/prop_m...,NaN,160660,160660,MARCELO THEODORO DE AGUIAR


Somando a quantidade de proposições por deputado

In [64]:
df_prep.nomeCivil.value_counts()

CARLOS GOMES BEZERRA                     546
ANTONIO CARLOS DE MENDES THAME           468
DARCI POMPEO DE MATTOS                   436
JOÃO ALBERTO FRAGA SILVA                 417
RÔMULO JOSÉ DE GOUVEIA                   306
CLEBER VERDE CORDEIRO MENDES             286
ARNALDO FARIA DE SÁ                      285
CARLOS ALBERTO CAVALCANTE DE SOUZA       275
RUBENS BUENO                             257
VALDIR COLATTO                           251
LUIZ CARLOS JORGE HAULY                  244
LINCOLN DINIZ PORTELA                    206
MARIA LAURA MONTEZA DE SOUZA CARNEIRO    201
CELSO UBIRAJARA RUSSOMANNO               201
LUIZ GONZAGA PATRIOTA                    200
JOÃO SANDES JÚNIOR                       200
FELIPE LEONE BORNIER DE OLIVEIRA         184
OTAVIO SANTOS SILVA LEITE                183
WELITON FERNANDES PRADO                  173
JAIR MESSIAS BOLSONARO                   170
VENEZIANO VITAL DO RÊGO SEGUNDO NETO     157
FLAVIO ALVES SABINO                      154
CARLOS HUM

Verificando a quantidade de proposições feita pelo depudado '74847'


In [51]:
'MESSIAS' in df_prep.nomeCivil

False

Proposições por ano do deputado '74847'

In [10]:
df[df.idAutor == '74847'].ano.value_counts().sort_index()

1991     9
1992     7
1993     7
1994     5
1995     5
1996    15
1997    12
1998     4
2000     6
2001     2
2002     4
2003     2
2004     1
2005     7
2006    13
2007    10
2008     6
2009     7
2010     4
2011     1
2012     1
2013     8
2014    11
2015    10
2016     7
2017     6
Name: ano, dtype: int64